In [1]:
from datetime import datetime
import math
import time
import tensorflow as tf
import os
import torch
from torch.autograd import Variable
import numpy
import gc
import sys

In [2]:
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [3]:
path = 'E:\\GPImage\\HandleImage\\Train_Enhance_Image\\'

In [4]:
import numpy as np
import cv2 as cv

# 读取图片， 并将data和label分别写入
def read_img(path):
    imgs = []
    labels = []
    fileList = os.listdir(path)
    totalNum = len(fileList)
    
    for i in range(0, totalNum):
        iPath = path + str(fileList[i])
        #  添加 文件标签
        # 获取文件名
        y = fileList[i]
        #     print(y)
    
        # 去除文件后缀
        y = os.path.splitext(y)
        #     print(y)
        y_ = y[0]
        #     print(y_)
    
        # 去除文件中的数字，获取文件标签
        # 设置标签 1为eye 2为back 3为face 4为left 5为up         
        y_ = "".join(filter(str.isalpha, y_))
        # print(type(y_))
        if(y_ == 'eye'):
            y_label = 1.0
        elif(y_ == 'back'):
            y_label = 2.0
        elif(y_ == 'face'):
            y_label = 3.0
        elif(y_ == 'left'):
            y_label = 4.0
        else:
            y_label = 5.0
        
        # 添加标签  
        labels.append(y_label)
    
        #   添加文件图片数据 
        src = cv.imread(iPath)
        image = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
        imgs.append(image)
    return np.asarray(imgs, np.float32), np.asarray(labels, np.float32)

In [5]:
def conv_op(input_op, name, kh, kw, n_out, dh, dw, p):
    print("卷积层")
    print("input_op的类型： " + str(type(input_op)))
    n_in = tf.Tensor.get_shape(input_op)[-1]
    #input_op.size()[-1]
    
#     n_in = input_op.get_shape()[-1].value
    print("n_in的大小 ： " + str(n_in))
#     print("P的类型" + str(type(p)))
    
    with tf.name_scope(name) as scope:
        kernel = tf.get_variable(scope+"w", shape=[kh,kw,n_in,n_out], dtype=tf.float32,
                                 initializer=tf.contrib.layers.xavier_initializer_conv2d())
#         print("kernel " + str(type(kernel)))
        conv = tf.nn.conv2d(input_op, kernel, (1, dh, dw, 1), padding='SAME')
        bias_init_val = tf.constant(0.0, shape=[n_out], dtype=tf.float32)
        biases = tf.Variable(bias_init_val, trainable=True, name='b')
        z = tf.nn.bias_add(conv,biases)
        activation = tf.nn.relu(z, name=scope)
        p += [kernel, biases]
        return activation

In [6]:
def fc_op(input_op, name, n_out, p):
    n_in = tf.Tensor.get_shape(input_op)[-1]
    
    with tf.name_scope(name) as scope:
        kernel = tf.get_variable(scope+"w", shape=[n_in, n_out], dtype=tf.float32,
                                 initializer=tf.contrib.layers.xavier_initializer())
        biases = tf.Variable(tf.constant(0.1, shape=[n_out], dtype=tf.float32), name='b')
        activation = tf.nn.relu_layer(input_op, kernel, biases, name= scope)
        p += [kernel, biases]
        return activation

In [7]:
def mpool_op(input_op, name, kh, kw, dh, dw):
    print("池化层")
    return tf.nn.max_pool(input_op,
                          ksize=[1, kh, kw, 1],
                          strides=[1, dh, dw, 1],
                          padding='SAME',
                          name=name)

In [8]:
def inference_op(input_op,keep_prob):
    
    p = []
    print("p 的类型" + str(type(p)))
    
    # 第一段卷积的第一个卷积层 卷积核3*3，共64个卷积核（输出通道数），步长1*1
    # input_op：256*256*1 输出尺寸256*256*64
    conv1_1 = conv_op(input_op, name="conv1_1", kh=3, kw=3, n_out=64, dh=1,
                      dw=1, p=p)
    print("conv1_1 " + str(conv1_1))
    # 第一段卷积的第2个卷积层 卷积核3*3，共64个卷积核（输出通道数），步长1*1
    # input_op：256*256*64 输出尺寸256*256*64
    conv1_2 = conv_op(conv1_1, name="conv1_2", kh=3, kw=3, n_out=64, dh=1,
                      dw=1, p=p)
    print("conv1_2 " + str(conv1_2))
    # 第一段卷积的pooling层，核2*2，步长2*2
    # input_op：256*256*64 输出尺寸128*128*64
    pool1 = mpool_op(conv1_2, name="pool1", kh=2, kw=2, dh=2, dw=2)
    print("pool1 " + str(pool1))
    
    # 下面是第2段卷积，包含2个卷积层和一个pooling层
    # 第2段卷积的第一个卷积层 卷积核3*3，共128个卷积核（输出通道数），步长1*1
    # input_op：128*128*64 输出尺寸128*128*128
    conv2_1 = conv_op(pool1, name="conv2_1", kh=3, kw=3, n_out=128, dh=1,
                      dw=1, p=p)
    print("conv2_1 " + str(conv2_1))
    # input_op：128*128*128 输出尺寸128*128*128
    conv2_2 = conv_op(conv2_1, name="conv2_2", kh=3, kw=3, n_out=128, dh=1,
                      dw=1, p=p)
    print("conv2_2 " + str(conv2_2))
    # input_op：128*128*128 输出尺寸64*64*128
    pool2 = mpool_op(conv2_2, name="pool2", kh=2, kw=2, dh=2, dw=2)
    print("pool2 " + str(pool2))
    
    # 下面是第3段卷积，包含3个卷积层和一个pooling层
    # 第3段卷积的第一个卷积层 卷积核3*3，共256个卷积核（输出通道数），步长1*1
    # input_op：64*64*128 输出尺寸64*64*256
    conv3_1 = conv_op(pool2, name="conv3_1", kh=3, kw=3, n_out=256, dh=1,
                      dw=1, p=p)
    print("conv3_1 " + str(conv3_1))
    # input_op：64*64*256 输出尺寸64*64*256
    conv3_2 = conv_op(conv3_1, name="conv3_2", kh=3, kw=3, n_out=256, dh=1,
                      dw=1, p=p)
    print("conv3_2 " + str(conv3_2))
    # input_op：64*64*256 输出尺寸64*64*256
    conv3_3 = conv_op(conv3_2, name="conv3_3", kh=3, kw=3, n_out=256, dh=1,
                      dw=1, p=p)
    print("conv3_3 " + str(conv3_3))
    # input_op：64*64*256 输出尺寸 32*32*256
    pool3 = mpool_op(conv3_3, name="pool3", kh=2, kw=2, dh=2, dw=2)
    print("pool3 " + str(pool3))
    
    
    # 下面是第4段卷积，包含3个卷积层和一个pooling层
    # 第3段卷积的第一个卷积层 卷积核3*3，共512个卷积核（输出通道数），步长1*1
    # input_op：32*32*256 输出尺寸32*32*512
    conv4_1 = conv_op(pool3, name="conv4_1", kh=3, kw=3, n_out=512, dh=1,
                      dw=1, p=p)
    print("conv4_1 " + str(conv4_1))
    # input_op：32*32*512 输出尺寸 32*32*512
    conv4_2 = conv_op(conv4_1, name="conv4_2", kh=3, kw=3, n_out=512, dh=1,
                      dw=1, p=p)
    print("conv4_2 " + str(conv4_2))
    # input_op：32*32*512 输出尺寸 32*32*512
    conv4_3 = conv_op(conv4_2, name="conv4_3", kh=3, kw=3, n_out=512, dh=1,
                      dw=1, p=p)
    print("conv4_3 " + str(conv4_3))
    # input_op：32*32*512 输出尺寸 16*16*512
    pool4 = mpool_op(conv4_3, name="pool4", kh=2, kw=2, dh=2, dw=2)
    print("pool4 " + str(pool4))
    
    # 前面4段卷积发现，VGG16每段卷积都是把图像面积变为1/4，但是通道数翻倍
    # 因此图像tensor的总尺寸缩小一半

    # 下面是第5段卷积，包含3个卷积层和一个pooling层
    # 第3段卷积的第一个卷积层 卷积核3*3，共512个卷积核（输出通道数），步长1*1
    # input_op：16*16*512 输出尺寸 16*16*512
    conv5_1 = conv_op(pool4, name="conv5_1", kh=3, kw=3, n_out=512, dh=1,
                      dw=1, p=p)
    print("conv5_1 " + str(conv5_1))
    # input_op：16*16*512 输出尺寸 16*16*512
    conv5_2 = conv_op(conv5_1, name="conv5_2", kh=3, kw=3, n_out=512, dh=1,
                      dw=1, p=p)
    # input_op：16*16*512 输出尺寸 16*16*512
    print("conv5_2 " + str(conv5_2))
    conv5_3 = conv_op(conv5_2, name="conv5_3", kh=3, kw=3, n_out=512, dh=1,
                      dw=1, p=p)
    print("conv5_3 " + str(conv5_3))
    # input_op：16*16*512 输出尺寸 8*8*512
    pool5 = mpool_op(conv5_3, name="pool5", kh=2, kw=2, dh=2, dw=2)
    print("pool5 " + str(pool5))
    
    # 将第五段卷积网络的结果扁平化
    # reshape将每张图片变为 8*8*512 = 32768的一维向量
    shp = pool5.get_shape()
    
    print("shp的shape: " + str(shp))
    
    flattened_shape = shp[1].value * shp[2].value * shp[3].value
    # tf.reshape(tensor, shape, name=None) 将tensor变换为参数shape的形式。
    resh1 = tf.reshape(pool5, [-1, flattened_shape], name="resh1")

    print("resh1的类型" + str(type(resh1)))
    print(resh1)
    
    # 第一个全连接层，是一个隐藏节点数为4096的全连接层
    # 后面接一个dropout层，训练时保留率为0.5，预测时为1.0
    fc6 = fc_op(resh1, name="fc6", n_out=4096, p=p)
    print("fc6 的shape: " + str(fc6))
    print("keep_prob 的shape： " + str(keep_prob))
    fc6_drop = tf.nn.dropout(fc6, keep_prob, name="fc6_drop")
 
    
    # 第2个全连接层，是一个隐藏节点数为4096的全连接层
    # 后面接一个dropout层，训练时保留率为0.5，预测时为1.0
    fc7 = fc_op(fc6_drop, name="fc7", n_out=4096, p=p)
    fc7_drop = tf.nn.dropout(fc7, keep_prob, name="fc7_drop")

    
    # 最后是一个1000个输出节点的全连接层
    # 利用softmax输出分类概率
    # argmax输出概率最大的类别
    fc8 = fc_op(fc7_drop, name="fc8", n_out=1000, p=p)
    softmax = tf.nn.softmax(fc8)
    predictions = tf.argmax(softmax, 1)
    
#     
    
    return predictions, softmax, fc8, p

In [9]:
def time_tensorflow_run(session, target, feed, info_string): # 与AlexNet非常相似，session参数一点点区别
    # num_steps_burn_in预热轮数，前几轮有显存加载，可以跳过时间评测
    # 只计算10轮迭代后的计算时间
    num_steps_burn_in = 10
    # 总时间
    total_duration = 0.0
    # 总时间的平方
    total_duration_squared = 0.0
    # 进行num_batches + num_steps_burn_in次迭代计算
    for i in range(num_batches + num_steps_burn_in):
        # 使用time.time()记录时间
        start_time = time.time()
        # 每次迭代通过session.run执行
        _ = session.run(target, feed_dict=feed) # 引入feed_dict方便后面传入keep_prob来控制Dropout层的保留比率
        duration = time.time() - start_time
        print(str(type(duration)) + "   time:  " + str(duration))
        # 在预热之后
        if i >= num_steps_burn_in:
            # 每10轮迭代显示当前迭代所需时间
            if not i % 10:
                print (str(datetime.now()) + ": step " +
                       str(i - num_steps_burn_in) + 
                       ", duration = " + str(duration))
            total_duration += duration
            total_duration_squared += duration * duration
    # 计算每轮迭代的平均耗时和标准差
    mn = total_duration / num_batches
    vr = total_duration_squared / num_batches - mn * mn
    sd = math.sqrt(vr)
    print (str(datetime.now()) + ": " +
           str(info_string) + "across " + 
           str(num_batches) + " steps, " +  
           str(mn) + " +/- " + str(sd) + "sec / batch")

In [10]:
def run_benchmark():
    
    with tf.Graph().as_default():
        
        image, label = read_img(path)
        print(image.shape)
        images_ = numpy.reshape(image, [-1, 1, 256, 256])
        print(images_.shape)
#         -----------------------------------------------------------
        print(str(type(images_)))# numpy
        print("***********************************************************")    
        in_put = Variable(torch.from_numpy(images_))
        print("run_benchmark中    image  " +str(type(in_put)))
        print("***********************************************************")
#         ----------------------------------------------------------- 
        
        print("元素的类型： " + str(type(label[-1])))
        
        keep_prob = tf.placeholder(tf.float32)
#         keep_prob = tf.convert_to_tensor(label)
#         keep_prob = Variable(torch.from_numpy(label))
#         print("标签的类型 ：" + str(type(keep_prob)))
        # keep_prob = tf.placeholder(tf.float32)
        
        predictions, softmax, fc8, p = inference_op(in_put, keep_prob)  # 构建网络结构获得参数列表

        print("...................................................")
        print("变量所占内存大小 ： " )
        print(sys.getsizeof(predictions))
        print(sys.getsizeof(softmax))
        print(sys.getsizeof(fc8))
        print(sys.getsizeof(p))
        print("...................................................")
        # 通过tf.session创建新的session，并通过global_variable初始化所有参数
        # 并通过run运行该session 只有运行run并feed数据运算才真正执行
        init = tf.global_variables_initializer()  # 初始化全局参数
        sess = tf.Session()  # 创建session
        sess.run(init)

        time_tensorflow_run(sess, predictions, {keep_prob: 1.0}, "Forward")  # 预测时节点保留率

        print("函数已大致运行完成///////////////////////////////////")
        print('计算VGGNet-16最后的全连接层的输出fc8的L2 loss')
        objective = tf.nn.l2_loss(fc8)  # 计算VGGNet-16最后的全连接层的输出fc8的L2 loss
        print('使用tf.gradients求相对于这个loss的所有模型参数的梯度')
        grad = tf.gradients(objective, p)  # 使用tf.gradients求相对于这个loss的所有模型参数的梯度
        print('求解梯度的操作grad')
        time_tensorflow_run(sess, grad, {keep_prob: 0.5}, "Forward-backward")  # 这里的target为求解梯度的操作grad
        print("运行完毕")

In [11]:
batch_size = 16  # VGGNet-16模型的体积较大，如果使用较大的batch_size，GPU显存会不够用
num_batches = 100
run_benchmark()

(766, 256, 256)
(766, 1, 256, 256)
<class 'numpy.ndarray'>
***********************************************************
run_benchmark中    image  <class 'torch.Tensor'>
***********************************************************
元素的类型： <class 'numpy.float32'>
p 的类型<class 'list'>
卷积层
input_op的类型： <class 'torch.Tensor'>
n_in的大小 ： 256
conv1_1 Tensor("conv1_1:0", shape=(766, 1, 256, 64), dtype=float32)
卷积层
input_op的类型： <class 'tensorflow.python.framework.ops.Tensor'>
n_in的大小 ： 64
conv1_2 Tensor("conv1_2:0", shape=(766, 1, 256, 64), dtype=float32)
池化层
pool1 Tensor("pool1:0", shape=(766, 1, 128, 64), dtype=float32)
卷积层
input_op的类型： <class 'tensorflow.python.framework.ops.Tensor'>
n_in的大小 ： 64
conv2_1 Tensor("conv2_1:0", shape=(766, 1, 128, 128), dtype=float32)
卷积层
input_op的类型： <class 'tensorflow.python.framework.ops.Tensor'>
n_in的大小 ： 128
conv2_2 Tensor("conv2_2:0", shape=(766, 1, 128, 128), dtype=float32)
池化层
pool2 Tensor("pool2:0", shape=(766, 1, 64, 128), dtype=float32)
卷积层
input_op的类型： <cl

<class 'float'>   time:  0.9434800148010254
<class 'float'>   time:  0.9374938011169434
<class 'float'>   time:  0.9374949932098389
<class 'float'>   time:  0.9454700946807861
2020-03-19 17:17:28.461071: Forwardacross 100 steps, 0.9370957493782044 +/- 0.004132867398004191sec / batch
函数已大致运行完成///////////////////////////////////
计算VGGNet-16最后的全连接层的输出fc8的L2 loss
使用tf.gradients求相对于这个loss的所有模型参数的梯度
求解梯度的操作grad
<class 'float'>   time:  30.60780692100525
<class 'float'>   time:  2.7484309673309326
<class 'float'>   time:  2.7656116485595703
<class 'float'>   time:  2.7676029205322266
<class 'float'>   time:  2.752645492553711
<class 'float'>   time:  2.759626626968384
<class 'float'>   time:  2.7536447048187256
<class 'float'>   time:  2.7706289291381836
<class 'float'>   time:  2.768571376800537
<class 'float'>   time:  2.7716047763824463
<class 'float'>   time:  2.758629560470581
2020-03-19 17:18:26.853422: step 0, duration = 2.758629560470581
<class 'float'>   time:  2.7766151428222656
<cl